In [ ]:
import os
import json
import datetime
import requests
import pandas as pd
import numpy as np
import util.data_puller as api
import plotly.figure_factory as ff
import tensorflow as tf
from json import JSONEncoder

apikey = os.environ['API_KEY']
base_url = base_url = os.environ['API_BASE_URL'] + '/' + os.environ['API_CHANNEL'] + '/' + os.environ['API_F']

history_size = 60
start = str(datetime.datetime.now()).split('.')[0]
end = str(datetime.datetime(2020, 3, 1))

datetime_to = datetime.datetime.strptime(f"{datetime.datetime.now().date()} 23:59:59", "%Y-%m-%d %H:%M:%S")

# NOTE: Take the day before to ensure only completed days
datetime_to = datetime_to - datetime.timedelta(1)

# NOTE: Add 1 second to get the start of the next day so it matches history_size.
datetime_from = (datetime_to - datetime.timedelta(history_size)) + datetime.timedelta(seconds=1)

print(start, end)

In [ ]:
df = api.apicallv3(base_url, apikey, str(start), str(end), True, False)

In [ ]:
corrs = df.corr()

figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True
)

figure.show()

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(df.values)
dataset = dataset.map(lambda row: tf.cast(row, 'float32'))
dataset = dataset.window(history_size, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(history_size))
dataset = dataset.map(lambda window: (window[:,1:], window[-1,0]))

arr = list(dataset.as_numpy_iterator())
print(arr[0])

In [ ]:
len(arr[0])

In [ ]:
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

tcx = [[]]
tcx[0] = df.values

#print(tcx)

data = json.dumps({"signature_name":"serving_default", "instances":tcx}, cls=NumpyArrayEncoder)
#print(f"Data: {data}")
headers = {"content-type":"application/json"}

json_response = requests.post("http://172.22.0.2:8501/v1/models/predictor:predict", data=data, headers=headers)
print(f"Json text: {json_response.text}")
predictions = json.loads(json_response.text)['predictions']

single_prediction = round(float(predictions[0][0]))

date = datetime.datetime.now() + datetime.timedelta(single_prediction)
date = date.date()

print(date)

print(single_prediction)